## openpose生图

此文件的作用是使用openpose的controlnet，以源文件夹中的图片为参考，加上指定的prompt，生成新的图片。

In [1]:
import json
from urllib import request, parse
import random
import uuid
import urllib.request
import urllib.parse
import io
import os
import time
from PIL.PngImagePlugin import PngInfo
import hashlib
from urllib.error import URLError


## import json file

In [2]:
promptJson = 'img2img_api.json'

with open(promptJson) as f:
    prompt = json.load(f)

print(str(prompt))

{'3': {'inputs': {'seed': 958148975345641, 'steps': 35, 'cfg': 7, 'sampler_name': 'dpmpp_2m_sde', 'scheduler': 'karras', 'denoise': 0.3, 'model': ['44', 0], 'positive': ['6', 0], 'negative': ['7', 0], 'latent_image': ['54', 0]}, 'class_type': 'KSampler'}, '4': {'inputs': {'ckpt_name': 'fudukiMix_v15.safetensors'}, 'class_type': 'CheckpointLoaderSimple'}, '6': {'inputs': {'text': 'masterpiece, best quality, ultra high res, highres,best shadow, physics-based rendering, extremely delicate and beautiful,extremely detailed, amazing,vivid,realistic,Bokeh,(a Japanese girl:1.5),(female child:1.2),cute,round face,shining skin, petite,flat chest,naked,nsfw,thigh,(shaved pussy),anal sex,after sex,Orange hair, Double ponytail,glasses, green headband, necklace, black bracelet, thin waist, black and red stockings\nBREAK\nShe was in a cozy room with colorful balloons, wood chairs, cloth sofas. ', 'clip': ['44', 1]}, 'class_type': 'CLIPTextEncode'}, '7': {'inputs': {'text': 'embedding:negativeXL_D, ',

In [3]:
server_address = "127.0.0.1:8188"
client_id = str(uuid.uuid4())


In [4]:
def changeImageSize(width, height):
    # 最小的宽高为 896
    limited = 896
    minSize = min(width, height)

    # 缩放宽高，使得最窄的边等于limited
    if minSize > limited:
        scale = minSize / limited
        widthScale = int(width / scale)
        heightScale = int(height / scale)
    else:
        scale = limited / minSize
        widthScale = int(width * scale)
        heightScale = int(height * scale)

    # 将两个值都除以64，然后取整,也就是说边长必须是64的倍数
    widthScale = int(widthScale/64) * 64
    heightScale = int(heightScale/64) * 64

    return widthScale, heightScale

def checkImageExsit(imageDir, imageName):
    for file in os.listdir(imageDir):
        if file.startswith(imageName):
            return True
    return False


def queue_prompt(prompt):
    while True:
        try:
            p = {"prompt": prompt, "client_id": client_id}
            data = json.dumps(p).encode('utf-8')
            req = urllib.request.Request("http://{}/prompt".format(server_address), data=data)
            return json.loads(urllib.request.urlopen(req, timeout=5).read())
        except URLError:
            print("连接超时，正在重试...")

def get_history(prompt_id):
    with urllib.request.urlopen("http://{}/history/{}".format(server_address, prompt_id)) as response:
        return json.loads(response.read())

def get_image(filename, subfolder, folder_type):
    data = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    url_values = urllib.parse.urlencode(data)
    with urllib.request.urlopen("http://{}/view?{}".format(server_address, url_values)) as response:
        return response.read()

def handle_whitespace(string: str):
    return string.strip().replace("\n", " ").replace("\r", " ").replace("\t", " ")

def parse_name(ckpt_name):
    path = ckpt_name
    filename = path.split("/")[-1]
    filename = filename.split(".")[:-1]
    filename = ".".join(filename)
    return filename


def calculate_sha256(file_path):
    sha256_hash = hashlib.sha256()

    with open(file_path, "rb") as f:
        # Read the file in chunks to avoid loading the entire file into memory
        for byte_block in iter(lambda: f.read(4096), b""):
            sha256_hash.update(byte_block)

    return sha256_hash.hexdigest()

def get_images(ws, prompt):
    prompt_id = queue_prompt(prompt)['prompt_id']
    output_images = {}
    while True:
        out = ws.recv()
        if isinstance(out, str):
            message = json.loads(out)
            if message['type'] == 'executing':
                data = message['data']
                if data['node'] is None and data['prompt_id'] == prompt_id:
                    break #Execution is done
        else:
            continue #previews are binary data

    history = get_history(prompt_id)[prompt_id]
    for o in history['outputs']:
        for node_id in history['outputs']:
            node_output = history['outputs'][node_id]
            if 'images' in node_output:
                images_output = []
                for image in node_output['images']:
                    image_data = get_image(image['filename'], image['subfolder'], image['type'])
                    images_output.append(image_data)
                output_images[node_id] = images_output

    return output_images
    
def saveImages(result, imageDir,imageName,comment,previewImage=0): 
    metadata = PngInfo()
    metadata.add_text("parameters", comment)  
    k = 0 
    for node_id in result:    
        if k < previewImage:
            k += 1
            continue    
        for image_data in result[node_id]:            
            image = Image.open(io.BytesIO(image_data))
            imagePath = imageDir + "/" + imageName + f"_{k}" + ".png"
            image.save(imagePath,pnginfo=metadata) 
            k += 1
    

In [5]:
# 子文件夹
subDir = "anal"

# 原图文件夹
originalDir = f"e:/SharedDirectory/PoleDriver/训练用\批量图片下载器/{subDir}"
# 设置图片保存路径
imageDir = f"f:/Poledriver/remake/{subDir}"
# 如果文件夹不存在，创建文件夹
if not os.path.exists(imageDir):
    os.mkdir(imageDir)

In [6]:

from PIL import Image
from sdparsers import ParserManager
import websocket

ws = websocket.WebSocket()
ws.connect("ws://{}/ws?clientId={}".format(server_address, client_id))

batch_size = 1
denoise_list = [0.5,0.6,0.7,0.8]
# 遍历原图文件夹里所有的图片
for originalImag in os.listdir(originalDir):
        # 如果不是jpg或png文件，跳过
        if not (originalImag.endswith(".jpg") or originalImag.endswith(".png") or originalImag.endswith(".jpeg")):
            continue
        
        originalImagePath = originalDir + "/" + originalImag
        # 如果文件名包含_H或者_B，跳过
        if originalImag.find("_H") != -1 or originalImag.find("_B") != -1:
            print(f"{originalImag}包含_H或者_B,判定为mask文件，跳过")
            continue
        
        # 读取原图片名字，去掉后缀.png
        originalImagName = originalImag.split(".")[0]
        # 检测目标文件夹里是否已经存在该图片，如果存在，跳过
        if checkImageExsit(imageDir, originalImagName):
            print(f"{originalImagName}已经存在，跳过")
            continue  
        
        # 读取原图，以及其大小
        with Image.open(originalImagePath) as originalImage:
            originalImageWidth = originalImage.size[0]
            originalImageHeight = originalImage.size[1]
            parser_manager = ParserManager()
            prompt_data = parser_manager.parse(originalImage)
            # 读取原图的提示词，如果解析失败，使用默认提示词
            try:
                promt_string = prompt_data.prompts[0][0].value  
                prompt['6']['inputs']['text'] = promt_string
                negative_string = prompt_data.prompts[0][1].value
                prompt['7']['inputs']['text'] = negative_string
            except:
                print(f"原图片{originalImag}解析信息失败,使用默认提示词")
              
        
        width,height = changeImageSize(originalImageWidth, originalImageHeight)  
        print(f"原图名字：{originalImagName}。大小：{originalImageWidth}x{originalImageHeight}")
        print(f"缩放后的图片大小是{width}*{height}")
        megapixels = width / originalImageWidth
        seed = random.randint(0, 976242998978323)
        
        # 下面是确定实际生成图片的大小，种子，以及保存图片信息
        # 保存图片的信息使用的是38号节点，也就是Save Image w/Metadata节点 
        prompt['12']['inputs']['image'] = originalImagePath
        prompt['3']['inputs']['seed']= seed        
        prompt['50']['inputs']['megapixels'] = megapixels
        prompt['54']['inputs']['amount'] = batch_size

        steps = prompt['3']['inputs']['steps']
        cfg = prompt['3']['inputs']['cfg']
        sampler_name = prompt['3']['inputs']['sampler_name']
        scheduler = prompt['3']['inputs']['scheduler']
        positive = prompt['6']['inputs']['text']
        negative = prompt['7']['inputs']['text']
        
        modelname = prompt['4']['inputs']['ckpt_name']
        basemodelname = parse_name(modelname)        
        ckpt_path = "e:/sdwebui/webui/models/Stable-diffusion/" + modelname
        modelhash = calculate_sha256(ckpt_path)[:10]
        
        comment = f"{handle_whitespace(positive)}\nNegative prompt: {handle_whitespace(negative)}\nSteps: {steps}, Sampler: {sampler_name}{f'_{scheduler}' if scheduler != 'normal' else ''}, CFG Scale: {cfg}, Seed: {seed}, Size: {width}x{height}, Model hash: {modelhash}, Model: {basemodelname}, Version: ComfyUI"        
        
        # 测试不同的denoise的出图效果
        for index in range(len(denoise_list)):
            denoise = denoise_list[index]
            prompt['3']['inputs']['denoise'] = denoise
            image_name = f"{originalImagName}_{index}"
            # 连接api，生成并保存图片
            start_time = time.time()        
            images = get_images(ws, prompt)   
            saveImages(images, imageDir, image_name, comment=comment,previewImage=0)     
            end_time = time.time()
            print(f"耗时{end_time-start_time}秒")
        

原图片103499035_p4_master1200.jpg解析信息失败,使用默认提示词
原图名字：103499035_p4_master1200。大小：800x1200
缩放后的图片大小是896*1344
耗时180.4861295223236秒
耗时34.260908365249634秒
耗时35.7503981590271秒
耗时35.00329065322876秒
原图片103885368_p1_master1200.jpg解析信息失败,使用默认提示词
原图名字：103885368_p1_master1200。大小：800x1200
缩放后的图片大小是896*1344
耗时35.04054069519043秒
耗时34.29357099533081秒
耗时34.197038412094116秒
耗时34.4558470249176秒
原图片103885368_p2_master1200.jpg解析信息失败,使用默认提示词
原图名字：103885368_p2_master1200。大小：800x1200
缩放后的图片大小是896*1344
耗时35.042126417160034秒
耗时33.49616026878357秒
耗时34.3183159828186秒
耗时34.25485134124756秒
原图片107299472_p0_master1200.jpg解析信息失败,使用默认提示词
原图名字：107299472_p0_master1200。大小：900x1200
缩放后的图片大小是832*1152
耗时29.218316078186035秒
耗时28.367817401885986秒
耗时28.38165783882141秒
耗时28.40340304374695秒
原图片107299472_p1_master1200.jpg解析信息失败,使用默认提示词
原图名字：107299472_p1_master1200。大小：900x1200
缩放后的图片大小是832*1152
耗时29.04793930053711秒
耗时28.456489086151123秒
耗时27.74083399772644秒
耗时26.744781494140625秒
原图片107299472_p4_master1200.jpg解析信息失败,使用默认提示词
原图名字：107299